In [1]:
# 🎓 Welcome to Our Institute — Face Recognition Attendance with Email Notification

# 📦 Step 1: Install dependencies
!pip install opencv-python onnxruntime gTTS --quiet

# 📚 Step 2: Imports
import os, cv2, urllib.request, numpy as np, csv, io, smtplib, ssl
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive
from IPython.display import display, Javascript, Audio
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image
from datetime import datetime
import onnxruntime as ort
from gtts import gTTS
from email.message import EmailMessage

# 🔊 Voice function
def speak(text):
    tts = gTTS(text)
    tts.save("voice.mp3")
    display(Audio("voice.mp3", autoplay=True))

# ✉ Email function
def send_email(name, timestamp):
    try:
        msg = EmailMessage()
        msg['Subject'] = f'📥 Attendance Marked: {name}'
        msg['From'] = 'system@institute.com'
        msg['To'] = 'businessoftshirts@gmail.com'
        msg.set_content(f'Student: {name}\nTime: {timestamp}\n\nAttendance has been recorded.')

        smtp_server = 'smtp.gmail.com'
        smtp_port = 587
        sender_email = 'sam.makhani33@gmail.com                                                                               '
        app_password = 'hpfi jhnc eixw udzs'

        context = ssl.create_default_context()
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls(context=context)
            server.login(sender_email, app_password)
            server.send_message(msg)

        print("✅ Email sent to admin.")
    except Exception as e:
        print(f"⚠ Email failed: {e}")

# 🌟 Greeting
print("🎉 Welcome to Our Institute!")
print("📷 Face will be recognized automatically.\n")

# 🗂 Step 3: Mount Google Drive
drive.mount('/content/drive')
known_dir = '/content/drive/MyDrive/content'
attendance_path = '/content/drive/MyDrive/content/attendance.csv'

# 📥 Step 4: Download models if missing
arcface_url = "https://huggingface.co/FoivosPar/Arc2Face/resolve/main/arcface.onnx"
arcface_path = "/content/arcface.onnx"

if not os.path.exists(arcface_path):
    print("⬇ Downloading ArcFace model...")
    urllib.request.urlretrieve(arcface_url, arcface_path)

proto = 'deploy.prototxt'
caffemdl = 'res10_300x300_ssd_iter_140000.caffemodel'

if not os.path.exists(proto):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt", proto)
if not os.path.exists(caffemdl):
    urllib.request.urlretrieve("https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel", caffemdl)

# 🧠 Step 5: Load models
session = ort.InferenceSession(arcface_path)
face_net = cv2.dnn.readNetFromCaffe(proto, caffemdl)

def get_embedding(img_path, mark_face=True):
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠ Could not load image: {img_path}")
        return None
    h, w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300,300)), 1.0, (300,300), (104,117,123))
    face_net.setInput(blob)
    det = face_net.forward()
    if det.shape[2] == 0 or det[0,0,0,2] < 0.5:
        print(f"⚠ No clear face found in: {img_path}")
        return None
    x1, y1, x2, y2 = (det[0,0,0,3:7]*[w,h,w,h]).astype(int)

    if mark_face:
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        cv2.circle(img, center, radius=5, color=(0, 0, 255), thickness=-1)
        cv2.imwrite(img_path, img)

    face = cv2.resize(img[y1:y2, x1:x2], (112,112))
    rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB).astype(np.float32)
    norm = (rgb/255.0 - 0.5)/0.5
    inp = np.transpose(norm, (2,0,1))[np.newaxis,:]
    emb = session.run(None, {session.get_inputs()[0].name: inp})[0]
    return emb.flatten()

# 📂 Step 6: Load known faces
print("📁 Loading known student faces...\n")
known = {}
for f in os.listdir(known_dir):
    if f.lower().endswith(('.jpg', '.jpeg', '.png')):
        emb = get_embedding(os.path.join(known_dir, f))
        if emb is not None:
            name = os.path.splitext(f)[0]
            known[name] = emb
            print(f"✅ Loaded: {name}")

# 📸 Step 7: Auto-capture user image
def capture():
    js_code = '''
        (async function() {
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({ video: true });
            video.srcObject = stream;
            video.style.display = 'block';
            video.style.marginBottom = '10px';
            video.style.maxWidth = '100%';
            document.body.appendChild(video);
            await video.play();
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight);
            await new Promise(resolve => setTimeout(resolve, 5000));  // Wait 5 sec
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getTracks().forEach(track => track.stop());
            video.remove();
            return canvas.toDataURL('image/jpeg');
        })();
    '''
    data = eval_js(js_code)
    img = b64decode(data.split(',')[1])
    with open('/content/captured.jpg', 'wb') as f:
        f.write(img)

capture()
print("\n📸 Image captured successfully!\n")

# 🔍 Step 8: Identify face and mark attendance
cap_emb = get_embedding('/content/captured.jpg')
marked = False

if cap_emb is None:
    print("❌ No face detected. Try again in better lighting.")
else:
    print("🔍 Recognizing face...\n")
    for name, emb in known.items():
        sim = cosine_similarity([cap_emb], [emb])[0][0]
        print(f"🆚 Comparing with {name}: {sim*100:.2f}%")
        if sim >= 0.50:
            print(f"✅ Face matched with {name}. Marking attendance...\n")
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            row = [name, now]
            header = ['Name', 'Time']
            if not os.path.exists(attendance_path):
                with open(attendance_path, 'w', newline='') as f:
                    csv.writer(f).writerow(header)
            with open(attendance_path, 'a', newline='') as f:
                csv.writer(f).writerow(row)

            speak(f"Hi {name}, welcome! You have been marked present.")
            send_email(name, now)
            marked = True
            break
        else:
            print("⛔ Not a match. Checking next...\n")

if not marked:
    print("⚠ Face not recognized. Attendance not marked.")


^C


ModuleNotFoundError: No module named 'cv2'


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
    --------------------------------------- 0.3/10.7 MB ? eta -:--:--
    --------------------------------------- 0.3/10.7 MB ? eta -:--:--
    --------------------------------------- 0.3/10.7 MB ? eta -:--:--
    --------------------------------------- 0.3/10.7 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.7 MB 266.5 kB/s eta 0:00:38
   - -------------------------------------- 0.5/10.7 MB 266.5 kB/s eta 0:


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install opencv-python numpy scikit-learn onnxruntime gTTS

^C


Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.7.0-cp313-cp313-win_amd64.whl.metadata (14 kB)
  Using cached scipy-1.16.0-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   - -------

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\hp\\AppData\\Roaming\\Python\\Python313\\site-packages\\scipy\\optimize\\zeros.py'
Check the permissions.


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install streamlit opencv-python numpy onnxruntime gTTS python-dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [3]:
!pip install opencv-python numpy scikit-learn onnxruntime gTTS

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.7.0-cp313-cp313-win_amd64.whl.metadata (14 kB)
  Using cached scipy-1.16.0-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install secure-smtplib

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os
import cv2
import urllib.request
import numpy as np
import csv
import io
import smtplib
import ssl
from sklearn.metrics.pairwise import cosine_similarity


In [10]:
# 🎓 Welcome to Our Institute — Face Recognition Attendance with Email Notification

# 📦 Step 1: Install dependencies
!pip install opencv-python onnxruntime gTTS --quiet

# 📚 Step 2: Imports
import os, cv2, urllib.request, numpy as np, csv, io, smtplib, ssl
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, Javascript, Audio
from base64 import b64decode
from PIL import Image
from datetime import datetime
import onnxruntime as ort
from gtts import gTTS
from email.message import EmailMessage

# 🔊 Voice function
def speak(text):
    tts = gTTS(text)
    tts.save("voice.mp3")
    display(Audio("voice.mp3", autoplay=True))

# ✉ Email function
def send_email(name, timestamp):
    try:
        msg = EmailMessage()
        msg['Subject'] = f'📥 Attendance Marked: {name}'
        msg['From'] = 'system@institute.com'
        msg['To'] = 'businessoftshirts@gmail.com'
        msg.set_content(f'Student: {name}\nTime: {timestamp}\n\nAttendance has been recorded.')

        smtp_server = 'smtp.gmail.com'
        smtp_port = 587
        sender_email = 'sam.makhani33@gmail.com                                                                               '
        app_password = 'hpfi jhnc eixw udzs'

        context = ssl.create_default_context()
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls(context=context)
            server.login(sender_email, app_password)
            server.send_message(msg)

        print("✅ Email sent to admin.")
    except Exception as e:
        print(f"⚠ Email failed: {e}")

# 🌟 Greeting
print("🎉 Welcome to Our Institute!")
print("📷 Face will be recognized automatically.\n")

# 🗂 Step 3: Mount Google Drive

known_dir = '/content'
attendance_path = '/content/attendance.csv'

# 📥 Step 4: Download models if missing
arcface_url = "https://huggingface.co/FoivosPar/Arc2Face/resolve/main/arcface.onnx"
arcface_path = "/content/arcface.onnx"

if not os.path.exists(arcface_path):
    print("⬇ Downloading ArcFace model...")
    urllib.request.urlretrieve(arcface_url, arcface_path)

proto = 'deploy.prototxt'
caffemdl = 'res10_300x300_ssd_iter_140000.caffemodel'

if not os.path.exists(proto):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt", proto)
if not os.path.exists(caffemdl):
    urllib.request.urlretrieve("https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel", caffemdl)

# 🧠 Step 5: Load models
session = ort.InferenceSession(arcface_path)
face_net = cv2.dnn.readNetFromCaffe(proto, caffemdl)

def get_embedding(img_path, mark_face=True):
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠ Could not load image: {img_path}")
        return None
    h, w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300,300)), 1.0, (300,300), (104,117,123))
    face_net.setInput(blob)
    det = face_net.forward()
    if det.shape[2] == 0 or det[0,0,0,2] < 0.5:
        print(f"⚠ No clear face found in: {img_path}")
        return None
    x1, y1, x2, y2 = (det[0,0,0,3:7]*[w,h,w,h]).astype(int)

    if mark_face:
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        cv2.circle(img, center, radius=5, color=(0, 0, 255), thickness=-1)
        cv2.imwrite(img_path, img)

    face = cv2.resize(img[y1:y2, x1:x2], (112,112))
    rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB).astype(np.float32)
    norm = (rgb/255.0 - 0.5)/0.5
    inp = np.transpose(norm, (2,0,1))[np.newaxis,:]
    emb = session.run(None, {session.get_inputs()[0].name: inp})[0]
    return emb.flatten()

# 📂 Step 6: Load known faces
print("📁 Loading known student faces...\n")
known = {}
for f in os.listdir(known_dir):
    if f.lower().endswith(('.jpg', '.jpeg', '.png')):
        emb = get_embedding(os.path.join(known_dir, f))
        if emb is not None:
            name = os.path.splitext(f)[0]
            known[name] = emb
            print(f"✅ Loaded: {name}")

# 📸 Step 7: Auto-capture user image
def capture():
    js_code = '''
        (async function() {
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({ video: true });
            video.srcObject = stream;
            video.style.display = 'block';
            video.style.marginBottom = '10px';
            video.style.maxWidth = '100%';
            document.body.appendChild(video);
            await video.play();
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight);
            await new Promise(resolve => setTimeout(resolve, 5000));  // Wait 5 sec
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getTracks().forEach(track => track.stop());
            video.remove();
            return canvas.toDataURL('image/jpeg');
        })();
    '''
    data = eval_js(js_code)
    img = b64decode(data.split(',')[1])
    with open('/content/captured.jpg', 'wb') as f:
        f.write(img)

capture()
print("\n📸 Image captured successfully!\n")

# 🔍 Step 8: Identify face and mark attendance
cap_emb = get_embedding('/content/captured.jpg')
marked = False

if cap_emb is None:
    print("❌ No face detected. Try again in better lighting.")
else:
    print("🔍 Recognizing face...\n")
    for name, emb in known.items():
        sim = cosine_similarity([cap_emb], [emb])[0][0]
        print(f"🆚 Comparing with {name}: {sim*100:.2f}%")
        if sim >= 0.50:
            print(f"✅ Face matched with {name}. Marking attendance...\n")
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            row = [name, now]
            header = ['Name', 'Time']
            if not os.path.exists(attendance_path):
                with open(attendance_path, 'w', newline='') as f:
                    csv.writer(f).writerow(header)
            with open(attendance_path, 'a', newline='') as f:
                csv.writer(f).writerow(row)

            speak(f"Hi {name}, welcome! You have been marked present.")
            send_email(name, now)
            marked = True
            break
        else:
            print("⛔ Not a match. Checking next...\n")

if not marked:
    print("⚠ Face not recognized. Attendance not marked.")



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


🎉 Welcome to Our Institute!
📷 Face will be recognized automatically.

⬇ Downloading ArcFace model...


FileNotFoundError: [Errno 2] No such file or directory: '/content/arcface.onnx'

In [9]:
!pip install Pillow

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import urllib.request
import os

# Create content folder if it doesn't exist
os.makedirs('./content', exist_ok=True)

# Download ArcFace model
model_url = "https://huggingface.co/FoivosPar/Arc2Face/resolve/main/arcface.onnx"
model_path = "./content/arcface.onnx"

if not os.path.exists(model_path):
    print("Downloading ArcFace model...")
    urllib.request.urlretrieve(model_url, model_path)
    print("Model downloaded successfully!")
else:
    print("Model already exists at:", os.path.abspath(model_path))

Model already exists at: e:\sameer makhani\attendance\content\arcface.onnx


In [19]:
# 🎓 Welcome to Our Institute — Face Recognition Attendance with Email Notification

# 📦 Step 1: Install dependencies
!pip install opencv-python onnxruntime gTTS --quiet

# 📚 Step 2: Imports
import os, cv2, urllib.request, numpy as np, csv, io, smtplib, ssl
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, Javascript, Audio
from base64 import b64decode
from PIL import Image
from datetime import datetime
import onnxruntime as ort
from gtts import gTTS
from email.message import EmailMessage

# 🔊 Voice function
def speak(text):
    tts = gTTS(text)
    tts.save("voice.mp3")
    display(Audio("voice.mp3", autoplay=True))

# ✉ Email function
def send_email(name, timestamp):
    try:
        msg = EmailMessage()
        msg['Subject'] = f'📥 Attendance Marked: {name}'
        msg['From'] = 'system@institute.com'
        msg['To'] = 'businessoftshirts@gmail.com'
        msg.set_content(f'Student: {name}\nTime: {timestamp}\n\nAttendance has been recorded.')

        smtp_server = 'smtp.gmail.com'
        smtp_port = 587
        sender_email = 'sam.makhani33@gmail.com                                                                               '
        app_password = 'hpfi jhnc eixw udzs'

        context = ssl.create_default_context()
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls(context=context)
            server.login(sender_email, app_password)
            server.send_message(msg)

        print("✅ Email sent to admin.")
    except Exception as e:
        print(f"⚠ Email failed: {e}")

# 🌟 Greeting
print("🎉 Welcome to Our Institute!")
print("📷 Face will be recognized automatically.\n")

# 🗂 Step 3: Mount Google Drive

known_dir = '/content'
attendance_path = '/content/attendance.csv'

# 📥 Step 4: Download models if missing
arcface_url = "https://huggingface.co/FoivosPar/Arc2Face/resolve/main/arcface.onnx"
arcface_path = "./content/arcface.onnx"

if not os.path.exists(arcface_path):
    print("⬇ Downloading ArcFace model...")
    urllib.request.urlretrieve(arcface_url, arcface_path)

proto = 'deploy.prototxt'
caffemdl = 'res10_300x300_ssd_iter_140000.caffemodel'

if not os.path.exists(proto):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt", proto)
if not os.path.exists(caffemdl):
    urllib.request.urlretrieve("https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel", caffemdl)

# 🧠 Step 5: Load models
session = ort.InferenceSession(arcface_path)
face_net = cv2.dnn.readNetFromCaffe(proto, caffemdl)

def get_embedding(img_path, mark_face=True):
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠ Could not load image: {img_path}")
        return None
    h, w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300,300)), 1.0, (300,300), (104,117,123))
    face_net.setInput(blob)
    det = face_net.forward()
    if det.shape[2] == 0 or det[0,0,0,2] < 0.5:
        print(f"⚠ No clear face found in: {img_path}")
        return None
    x1, y1, x2, y2 = (det[0,0,0,3:7]*[w,h,w,h]).astype(int)

    if mark_face:
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        cv2.circle(img, center, radius=5, color=(0, 0, 255), thickness=-1)
        cv2.imwrite(img_path, img)

    face = cv2.resize(img[y1:y2, x1:x2], (112,112))
    rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB).astype(np.float32)
    norm = (rgb/255.0 - 0.5)/0.5
    inp = np.transpose(norm, (2,0,1))[np.newaxis,:]
    emb = session.run(None, {session.get_inputs()[0].name: inp})[0]
    return emb.flatten()

# 📂 Step 6: Load known faces
print("📁 Loading known student faces...\n")
known = {}
for f in os.listdir(known_dir):
    if f.lower().endswith(('.jpg', '.jpeg', '.png')):
        emb = get_embedding(os.path.join(known_dir, f))
        if emb is not None:
            name = os.path.splitext(f)[0]
            known[name] = emb
            print(f"✅ Loaded: {name}")

# 📸 Step 7: Auto-capture user image
def capture():
    js_code = '''
        (async function() {
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({ video: true });
            video.srcObject = stream;
            video.style.display = 'block';
            video.style.marginBottom = '10px';
            video.style.maxWidth = '100%';
            document.body.appendChild(video);
            await video.play();
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight);
            await new Promise(resolve => setTimeout(resolve, 5000));  // Wait 5 sec
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getTracks().forEach(track => track.stop());
            video.remove();
            return canvas.toDataURL('image/jpeg');
        })();
    '''
    data = eval_js(js_code)
    img = b64decode(data.split(',')[1])
    with open('/content/captured.jpg', 'wb') as f:
        f.write(img)

capture()
print("\n📸 Image captured successfully!\n")

# 🔍 Step 8: Identify face and mark attendance
cap_emb = get_embedding('/content/captured.jpg')
marked = False

if cap_emb is None:
    print("❌ No face detected. Try again in better lighting.")
else:
    print("🔍 Recognizing face...\n")
    for name, emb in known.items():
        sim = cosine_similarity([cap_emb], [emb])[0][0]
        print(f"🆚 Comparing with {name}: {sim*100:.2f}%")
        if sim >= 0.50:
            print(f"✅ Face matched with {name}. Marking attendance...\n")
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            row = [name, now]
            header = ['Name', 'Time']
            if not os.path.exists(attendance_path):
                with open(attendance_path, 'w', newline='') as f:
                    csv.writer(f).writerow(header)
            with open(attendance_path, 'a', newline='') as f:
                csv.writer(f).writerow(row)

            speak(f"Hi {name}, welcome! You have been marked present.")
            send_email(name, now)
            marked = True
            break
        else:
            print("⛔ Not a match. Checking next...\n")

if not marked:
    print("⚠ Face not recognized. Attendance not marked.")


🎉 Welcome to Our Institute!
📷 Face will be recognized automatically.




[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


InvalidProtobuf: [ONNXRuntimeError] : 7 : INVALID_PROTOBUF : Load model from ./content/arcface.onnx failed:Protobuf parsing failed.

In [13]:
import os
model_path = "./content/arcface.onnx"
if os.path.exists(model_path):
    os.remove(model_path)
    print("Removed corrupted model file")

Removed corrupted model file


In [16]:
import urllib.request
import ssl

# Create a SSL context to avoid certificate issues
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

model_urls = [
    "https://github.com/onnx/models/raw/main/vision/body_analysis/arcface/model/arcfaceresnet100-8.onnx",
    "https://huggingface.co/FoivosPar/Arc2Face/resolve/main/arcface.onnx"
]

for model_url in model_urls:
    try:
        print(f"Trying to download from: {model_url}")
        urllib.request.urlretrieve(model_url, model_path, context=ssl_context)
        print("Download successful!")
        break
    except Exception as e:
        print(f"Failed to download from {model_url}: {str(e)}")
else:
    print("All download attempts failed")

Trying to download from: https://github.com/onnx/models/raw/main/vision/body_analysis/arcface/model/arcfaceresnet100-8.onnx
Failed to download from https://github.com/onnx/models/raw/main/vision/body_analysis/arcface/model/arcfaceresnet100-8.onnx: urlretrieve() got an unexpected keyword argument 'context'
Trying to download from: https://huggingface.co/FoivosPar/Arc2Face/resolve/main/arcface.onnx
Failed to download from https://huggingface.co/FoivosPar/Arc2Face/resolve/main/arcface.onnx: urlretrieve() got an unexpected keyword argument 'context'
All download attempts failed


In [18]:
import urllib.request
import os

model_urls = [
    "https://github.com/onnx/models/raw/main/vision/body_analysis/arcface/model/arcfaceresnet100-8.onnx",
    "https://huggingface.co/FoivosPar/Arc2Face/resolve/main/arcface.onnx",
    "https://storage.googleapis.com/ailia-models/arcface/arcfaceresnet100-8.onnx"  # Mirror
]

model_path = "./content/arcface.onnx"
os.makedirs("./content", exist_ok=True)

for model_url in model_urls:
    try:
        print(f"Downloading from {model_url}...")
        urllib.request.urlretrieve(model_url, model_path)
        print(f"Successfully downloaded to {model_path}")
        print(f"File size: {os.path.getsize(model_path)/1e6:.1f} MB")
        break
    except Exception as e:
        print(f"Failed: {str(e)}")
else:
    print("All download attempts failed. Trying manual solution...")

Failed: HTTP Error 404: Not Found


KeyboardInterrupt: 

In [20]:
import requests
import os

# Create content directory if needed
os.makedirs('./content', exist_ok=True)

# Download from reliable mirror with progress bar
def download_file(url, filename):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    total_size = int(response.headers.get('content-length', 0))
    
    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    return filename

# Try multiple verified sources
sources = [
    "https://storage.googleapis.com/ailia-models/arcface/arcfaceresnet100-8.onnx",
    "https://github.com/onnx/models/raw/main/vision/body_analysis/arcface/model/arcfaceresnet100-8.onnx"
]

for url in sources:
    try:
        print(f"Trying {url}")
        model_path = download_file(url, "./content/arcface.onnx")
        print(f"Downloaded to {model_path}")
        print(f"File size: {os.path.getsize(model_path)/1e6:.2f} MB")
        break
    except Exception as e:
        print(f"Failed: {str(e)}")
else:
    print("All sources failed. Trying manual solution...")

Trying https://storage.googleapis.com/ailia-models/arcface/arcfaceresnet100-8.onnx
Failed: 404 Client Error: Not Found for url: https://storage.googleapis.com/ailia-models/arcface/arcfaceresnet100-8.onnx
Trying https://github.com/onnx/models/raw/main/vision/body_analysis/arcface/model/arcfaceresnet100-8.onnx
Failed: 404 Client Error: Not Found for url: https://github.com/onnx/models/raw/main/vision/body_analysis/arcface/model/arcfaceresnet100-8.onnx
All sources failed. Trying manual solution...


In [ ]:
# 📦 Step 1: Install dependencies (run in terminal first)
# pip install opencv-python numpy onnxruntime gTTS scikit-learn Pillow python-dotenv

# 📚 Step 2: Imports
import os
import cv2
import numpy as np
import csv
import smtplib
import ssl
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import onnxruntime as ort
from gtts import gTTS
from email.message import EmailMessage
from PIL import Image
# import pygame  # For local audio playback
import time

# 🔊 Voice function (local version)
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("voice.mp3")
    
    # # Initialize pygame mixer
    # pygame.mixer.init()
    # pygame.mixer.music.load("voice.mp3")
    # pygame.mixer.music.play()
    # while pygame.mixer.music.get_busy():
    #     time.sleep(0.1)

# ✉ Email function (updated for local)
def send_email(name, timestamp):
    try:
        msg = EmailMessage()
        msg['Subject'] = f'📥 Attendance Marked: {name}'
        msg['From'] = 'system@institute.com'
        msg['To'] = 'businessoftshirts@gmail.com'
        msg.set_content(f'Student: {name}\nTime: {timestamp}\n\nAttendance has been recorded.')

        smtp_server = 'smtp.gmail.com'
        smtp_port = 587
        sender_email = 'sam.makhani33@gmail.com'
        app_password = 'hpfi jhnc eixw udzs'

        context = ssl.create_default_context()
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls(context=context)
            server.login(sender_email, app_password)
            server.send_message(msg)

        print("✅ Email sent to admin.")
    except Exception as e:
        print(f"⚠ Email failed: {e}")

# 🌟 Greeting
print("🎉 Welcome to Our Institute!")
print("📷 Face will be recognized automatically.\n")

# 🗂 Step 3: Local file paths
current_dir = os.path.dirname(os.path.abspath(__file__))
known_dir = os.path.join(current_dir, 'content', 'known_faces')
attendance_path = os.path.join(current_dir, 'content', 'attendance.csv')
os.makedirs(known_dir, exist_ok=True)

# 📥 Step 4: Download models if missing
arcface_path = os.path.join(current_dir, 'content', 'arcface.onnx')
proto = os.path.join(current_dir, 'deploy.prototxt')
caffemdl = os.path.join(current_dir, 'res10_300x300_ssd_iter_140000.caffemodel')

# 🧠 Step 5: Load models
try:
    session = ort.InferenceSession(arcface_path)
    face_net = cv2.dnn.readNetFromCaffe(proto, caffemdl)
except Exception as e:
    print(f"⚠ Model loading failed: {e}")
    print("Please ensure you have these files in your content folder:")
    print(f"- {arcface_path}")
    print(f"- {proto}")
    print(f"- {caffemdl}")
    exit()

# ... [rest of your functions remain the same until capture()]

# 📸 Step 7: Local camera capture
def capture():
    cap = cv2.VideoCapture(0)  # Use default camera
    
    print("Looking for camera...")
    if not cap.isOpened():
        print("❌ Cannot open camera")
        return None
        
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to capture frame")
        return None
        
    save_path = os.path.join(current_dir, 'content', 'captured.jpg')
    cv2.imwrite(save_path, frame)
    cap.release()
    return save_path

# 🔍 Step 8: Main execution
if __name__ == "__main__":
    # Load known faces
    known = {}
    print("📁 Loading known student faces...\n")
    for f in os.listdir(known_dir):
        if f.lower().endswith(('.jpg', '.jpeg', '.png')):
            emb = get_embedding(os.path.join(known_dir, f))
            if emb is not None:
                name = os.path.splitext(f)[0]
                known[name] = emb
                print(f"✅ Loaded: {name}")

    # Capture and process
    img_path = capture()
    if img_path:
        print("\n📸 Image captured successfully!\n")
        cap_emb = get_embedding(img_path)
        
        # ... [rest of your attendance marking logic]

🎉 Welcome to Our Institute!
📷 Face will be recognized automatically.



NameError: name '__file__' is not defined

In [25]:
# attendancecode.ipynb
# Face Recognition Attendance System - Local Version

# 📦 Step 1: Install dependencies (run this first in a cell)
# !pip install opencv-python numpy onnxruntime gTTS scikit-learn Pillow python-dotenv pygame

# 📚 Step 2: Imports
import os
import cv2
import numpy as np
import csv
import smtplib
import ssl
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import onnxruntime as ort
from gtts import gTTS
from email.message import EmailMessage
import pygame
import time
from PIL import Image

# 🔊 Voice function (local version)
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("voice.mp3")
    pygame.mixer.init()
    pygame.mixer.music.load("voice.mp3")
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        time.sleep(0.1)

# ✉ Email function
def send_email(name, timestamp):
    try:
        msg = EmailMessage()
        msg['Subject'] = f'📥 Attendance Marked: {name}'
        msg['From'] = 'system@institute.com'
        msg['To'] = 'businessoftshirts@gmail.com'
        msg.set_content(f'Student: {name}\nTime: {timestamp}\n\nAttendance has been recorded.')

        smtp_server = 'smtp.gmail.com'
        smtp_port = 587
        sender_email = 'sam.makhani33@gmail.com'
        app_password = 'hpfi jhnc eixw udzs'

        context = ssl.create_default_context()
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls(context=context)
            server.login(sender_email, app_password)
            server.send_message(msg)
        print("✅ Email sent to admin.")
    except Exception as e:
        print(f"⚠ Email failed: {e}")

# 🌟 Initialize system
print("🎉 Welcome to Our Institute!")
print("📷 Face will be recognized automatically.\n")

# 🗂 File paths configuration
BASE_DIR = os.getcwd()
CONTENT_DIR = os.path.join(BASE_DIR, 'content')
KNOWN_FACES_DIR = os.path.join(CONTENT_DIR, 'known_faces')
ATTENDANCE_PATH = os.path.join(CONTENT_DIR, 'attendance.csv')
ARCFACE_PATH = os.path.join(CONTENT_DIR, 'arcface.onnx')
PROTO_PATH = os.path.join(BASE_DIR, 'deploy.prototxt')
CAFFEMDL_PATH = os.path.join(BASE_DIR, 'res10_300x300_ssd_iter_140000.caffemodel')

# Ensure directories exist
os.makedirs(KNOWN_FACES_DIR, exist_ok=True)

# 🧠 Face recognition functions
def get_embedding(img_path, mark_face=True):
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠ Could not load image: {img_path}")
        return None
        
    h, w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300,300)), 1.0, (300,300), (104,117,123))
    face_net.setInput(blob)
    det = face_net.forward()
    
    if det.shape[2] == 0 or det[0,0,0,2] < 0.5:
        print(f"⚠ No clear face found in: {img_path}")
        return None
        
    x1, y1, x2, y2 = (det[0,0,0,3:7]*[w,h,w,h]).astype(int)

    if mark_face:
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        cv2.circle(img, center, radius=5, color=(0, 0, 255), thickness=-1)
        cv2.imwrite(img_path, img)

    face = cv2.resize(img[y1:y2, x1:x2], (112,112))
    rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB).astype(np.float32)
    norm = (rgb/255.0 - 0.5)/0.5
    inp = np.transpose(norm, (2,0,1))[np.newaxis,:]
    emb = session.run(None, {session.get_inputs()[0].name: inp})[0]
    return emb.flatten()

# 📸 Camera capture function
def capture_image():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Error: Could not open camera")
        return None
    
    print("Smile! Capturing image in 3 seconds...")
    time.sleep(3)
    
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print("❌ Failed to capture image")
        return None
    
    capture_path = os.path.join(CONTENT_DIR, 'captured.jpg')
    cv2.imwrite(capture_path, frame)
    return capture_path

# 🔍 Main recognition flow
try:
    # Load models
    session = ort.InferenceSession(ARCFACE_PATH)
    face_net = cv2.dnn.readNetFromCaffe(PROTO_PATH, CAFFEMDL_PATH)
    
    # Load known faces
    known_faces = {}
    print("📁 Loading known student faces...")
    for face_file in os.listdir(KNOWN_FACES_DIR):
        if face_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            face_path = os.path.join(KNOWN_FACES_DIR, face_file)
            embedding = get_embedding(face_path)
            if embedding is not None:
                name = os.path.splitext(face_file)[0]
                known_faces[name] = embedding
                print(f"✅ Loaded: {name}")
    
    # Capture new face
    print("\nStarting face capture...")
    captured_path = capture_image()
    
    if captured_path:
        print("\n🔍 Recognizing face...")
        captured_embedding = get_embedding(captured_path)
        
        if captured_embedding is not None:
            marked = False
            for name, known_embedding in known_faces.items():
                similarity = cosine_similarity([captured_embedding], [known_embedding])[0][0]
                print(f"🆚 Comparing with {name}: {similarity*100:.2f}%")
                
                if similarity >= 0.50:
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    print(f"\n✅ Match found: {name} ({(similarity*100):.2f}%)")
                    
                    # Record attendance
                    header = ['Name', 'Time']
                    row = [name, timestamp]
                    
                    if not os.path.exists(ATTENDANCE_PATH):
                        with open(ATTENDANCE_PATH, 'w', newline='') as f:
                            csv.writer(f).writerow(header)
                    
                    with open(ATTENDANCE_PATH, 'a', newline='') as f:
                        csv.writer(f).writerow(row)
                    
                    speak(f"Welcome {name}, attendance marked!")
                    send_email(name, timestamp)
                    marked = True
                    break
            
            if not marked:
                print("⚠ No matching face found")
                speak("Sorry, we couldn't recognize you")
        else:
            print("❌ No face detected in captured image")
    else:
        print("❌ Failed to capture image")

except Exception as e:
    print(f"⚠ System error: {str(e)}")
finally:
    print("\nSystem shutdown")

pygame 2.6.1 (SDL 2.28.4, Python 3.13.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
🎉 Welcome to Our Institute!
📷 Face will be recognized automatically.

⚠ System error: [ONNXRuntimeError] : 7 : INVALID_PROTOBUF : Load model from e:\sameer makhani\attendance\content\arcface.onnx failed:Protobuf parsing failed.

System shutdown


In [24]:
!pip install pygame opencv-python numpy onnxruntime gTTS scikit-learn Pillow python-dotenv

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.6 MB 650.3 kB/s eta 0:00:16
   - -------------------------------------- 0.5/10.6 MB 650.3 kB/s eta 0:00:16
   -- ------------------------------------- 0.8/10.6 MB 658.2 kB/s eta 0:00:15
   -- ------------------------------------- 0.8/10.6 MB 658.2 kB/s eta 0:00:15
   --- ------------------------------------ 1.0/10.6 MB 643.6 kB/s eta 0:00:15
   --- ------------------------------------ 1.0/10.6 MB 643.6 kB/s eta 0:00:15
   --- ------------------------------------ 1.0/10.6 MB 643.6 kB/s eta 0:00:15
   --- ------------------------------------ 1.0/10.6 MB 643.6 kB/s eta 0:00:15
   ----- -


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
# attendancecode.ipynb - Face Recognition Attendance System
# Complete Local Version for Your E:\Attendance Structure

# 1. INSTALL DEPENDENCIES (Run this first in a cell)
# !pip install opencv-python numpy onnxruntime gTTS scikit-learn Pillow python-dotenv

# 2. IMPORTS
import os
import cv2
import numpy as np
import csv
import smtplib
import ssl
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import onnxruntime as ort
from gtts import gTTS
from email.message import EmailMessage
import time
import platform

# 3. PATH CONFIGURATION (Updated for your E: drive)
BASE_DIR = r"E:\Attendance"  # Raw string for Windows path
CONTENT_DIR = os.path.join(BASE_DIR, 'content')
KNOWN_FACES_DIR = os.path.join(CONTENT_DIR, 'known_faces')
ATTENDANCE_PATH = os.path.join(CONTENT_DIR, 'attendance.csv')
ARCFACE_PATH = os.path.join(CONTENT_DIR, 'arcface.onnx')
PROTO_PATH = os.path.join(BASE_DIR, 'deploy.prototxt')
CAFFEMDL_PATH = os.path.join(BASE_DIR, 'res10_300x300_ssd_iter_140000.caffemodel')

# Create directories if missing
os.makedirs(KNOWN_FACES_DIR, exist_ok=True)

# 4. AUDIO FEEDBACK (Platform-independent)
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("voice.mp3")
    
    system = platform.system()
    if system == "Windows":
        os.system("start voice.mp3")
    elif system == "Darwin":  # Mac
        os.system("afplay voice.mp3")
    else:  # Linux
        os.system("mpg123 voice.mp3")
    time.sleep(2)  # Allow audio to play

# 5. EMAIL FUNCTION
def send_email(name, timestamp):
    try:
        msg = EmailMessage()
        msg['Subject'] = f'Attendance Marked: {name}'
        msg['From'] = 'system@institute.com'
        msg['To'] = 'businessoftshirts@gmail.com'
        msg.set_content(f'Student: {name}\nTime: {timestamp}')

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login('sam.makhani33@gmail.com', 'hpfi jhnc eixw udzs')
            server.send_message(msg)
        print("✅ Email sent")
    except Exception as e:
        print(f"⚠ Email failed: {e}")

# 6. FACE RECOGNITION CORE
def get_embedding(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠ Couldn't load image: {img_path}")
        return None
        
    # Face detection
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300,300)), 1.0, (300,300), (104,117,123))
    face_net.setInput(blob)
    detections = face_net.forward()
    
    if detections.shape[2] == 0 or detections[0,0,0,2] < 0.5:
        print(f"⚠ No face found in: {img_path}")
        return None
        
    # Extract face
    h, w = img.shape[:2]
    box = detections[0,0,0,3:7] * np.array([w,h,w,h])
    (x1, y1, x2, y2) = box.astype("int")
    face = img[y1:y2, x1:x2]
    
    # Get embedding
    face = cv2.resize(face, (112,112))
    rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB).astype(np.float32)
    norm = (rgb/255.0 - 0.5)/0.5
    inp = np.transpose(norm, (2,0,1))[np.newaxis,:]
    return session.run(None, {session.get_inputs()[0].name: inp})[0].flatten()

# 7. CAMERA CAPTURE
def capture_face():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Camera error")
        return None
    
    print("Smile! Capturing in 3...")
    time.sleep(3)
    
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print("❌ Capture failed")
        return None
    
    save_path = os.path.join(CONTENT_DIR, 'captured.jpg')
    cv2.imwrite(save_path, frame)
    return save_path

# 8. MAIN SYSTEM
print("🎉 Welcome to Face Recognition Attendance!")
print(f"Base directory: {BASE_DIR}\n")

try:
    # Load models
    session = ort.InferenceSession(ARCFACE_PATH)
    face_net = cv2.dnn.readNetFromCaffe(PROTO_PATH, CAFFEMDL_PATH)
    
    # Load known faces
    known = {}
    print("Loading known faces...")
    for file in os.listdir(KNOWN_FACES_DIR):
        if file.lower().endswith(('.jpg','.jpeg','.png')):
            emb = get_embedding(os.path.join(KNOWN_FACES_DIR, file))
            if emb is not None:
                known[os.path.splitext(file)[0]] = emb
                print(f"✅ Loaded: {file}")
    
    # Capture new face
    print("\nStarting face capture...")
    captured = capture_face()
    
    if captured:
        print("Processing image...")
        emb = get_embedding(captured)
        
        if emb is not None:
            # Find match
            for name, known_emb in known.items():
                similarity = cosine_similarity([emb], [known_emb])[0][0]
                print(f"{name}: {similarity*100:.1f}% match")
                
                if similarity >= 0.5:  # Threshold
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    
                    # Save to CSV
                    with open(ATTENDANCE_PATH, 'a', newline='') as f:
                        writer = csv.writer(f)
                        if os.stat(ATTENDANCE_PATH).st_size == 0:
                            writer.writerow(['Name','Time'])
                        writer.writerow([name, timestamp])
                    
                    speak(f"Welcome {name}, attendance marked!")
                    send_email(name, timestamp)
                    break
            else:
                print("⚠ No match found")
                speak("Sorry, not recognized")
        
except Exception as e:
    print(f"⚠ System error: {e}")
finally:
    print("\nSystem shutdown")

🎉 Welcome to Face Recognition Attendance!
Base directory: E:\Attendance

⚠ System error: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from E:\Attendance\content\arcface.onnx failed:Load model E:\Attendance\content\arcface.onnx failed. File doesn't exist

System shutdown


In [27]:
# attendancecode.ipynb - Face Recognition Attendance System
# FULLY UPDATED FOR YOUR E:\sameer makhani\attendance STRUCTURE

# 1. INSTALL DEPENDENCIES (Run this first)
# !pip install opencv-python numpy onnxruntime gTTS scikit-learn Pillow python-dotenv

# 2. IMPORTS
import os
import cv2
import numpy as np
import csv
import smtplib
import ssl
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import onnxruntime as ort
from gtts import gTTS
from email.message import EmailMessage
import time
import platform
import urllib.request

# 3. PATH CONFIGURATION (UPDATED FOR YOUR STRUCTURE)
BASE_DIR = r"E:\sameer makhani\attendance"  # RAW STRING FOR WINDOWS
CONTENT_DIR = os.path.join(BASE_DIR, 'content')
KNOWN_FACES_DIR = os.path.join(CONTENT_DIR, 'known_faces')
ATTENDANCE_PATH = os.path.join(CONTENT_DIR, 'attendance.csv')
ARCFACE_PATH = os.path.join(CONTENT_DIR, 'arcface.onnx')
PROTO_PATH = os.path.join(BASE_DIR, 'deploy.prototxt')
CAFFEMDL_PATH = os.path.join(BASE_DIR, 'res10_300x300_ssd_iter_140000.caffemodel')

# 4. VERIFY FILES EXIST
def verify_files():
    required_files = {
        "ArcFace Model": ARCFACE_PATH,
        "Prototxt File": PROTO_PATH,
        "Caffe Model": CAFFEMDL_PATH,
        "Known Faces Folder": KNOWN_FACES_DIR
    }
    
    print("🔍 Verifying Files:")
    all_exists = True
    for name, path in required_files.items():
        exists = os.path.exists(path)
        print(f"{'✅' if exists else '❌'} {name}: {path}")
        if not exists: all_exists = False
    
    if not all_exists:
        print("\n⚠ Missing files detected. Attempting to download models...")
        download_models()

def download_models():
    # Download ArcFace model if missing
    if not os.path.exists(ARCFACE_PATH):
        print("Downloading ArcFace model...")
        try:
            urllib.request.urlretrieve(
                "https://storage.googleapis.com/ailia-models/arcface/arcfaceresnet100-8.onnx",
                ARCFACE_PATH
            )
            print(f"✅ ArcFace model saved to {ARCFACE_PATH}")
        except Exception as e:
            print(f"❌ Download failed: {e}")

    # Download Caffe files if missing
    if not os.path.exists(PROTO_PATH):
        print("Downloading deploy.prototxt...")
        urllib.request.urlretrieve(
            "https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt",
            PROTO_PATH
        )
    
    if not os.path.exists(CAFFEMDL_PATH):
        print("Downloading Caffe model...")
        urllib.request.urlretrieve(
            "https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel",
            CAFFEMDL_PATH
        )

# 5. AUDIO FEEDBACK
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("voice.mp3")
    
    system = platform.system()
    if system == "Windows":
        os.system("start voice.mp3")
    elif system == "Darwin":  # Mac
        os.system("afplay voice.mp3")
    else:  # Linux
        os.system("mpg123 voice.mp3")
    time.sleep(2)  # Allow audio to play

# 6. EMAIL FUNCTION
def send_email(name, timestamp):
    try:
        msg = EmailMessage()
        msg['Subject'] = f'Attendance Marked: {name}'
        msg['From'] = 'system@institute.com'
        msg['To'] = 'businessoftshirts@gmail.com'
        msg.set_content(f'Student: {name}\nTime: {timestamp}')

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login('sam.makhani33@gmail.com', 'hpfi jhnc eixw udzs')
            server.send_message(msg)
        print("✅ Email sent")
    except Exception as e:
        print(f"⚠ Email failed: {e}")

# 7. FACE RECOGNITION CORE
def get_embedding(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠ Couldn't load image: {img_path}")
        return None
        
    # Face detection
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300,300)), 1.0, (300,300), (104,117,123))
    face_net.setInput(blob)
    detections = face_net.forward()
    
    if detections.shape[2] == 0 or detections[0,0,0,2] < 0.5:
        print(f"⚠ No face found in: {img_path}")
        return None
        
    # Extract face
    h, w = img.shape[:2]
    box = detections[0,0,0,3:7] * np.array([w,h,w,h])
    (x1, y1, x2, y2) = box.astype("int")
    face = img[y1:y2, x1:x2]
    
    # Get embedding
    face = cv2.resize(face, (112,112))
    rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB).astype(np.float32)
    norm = (rgb/255.0 - 0.5)/0.5
    inp = np.transpose(norm, (2,0,1))[np.newaxis,:]
    return session.run(None, {session.get_inputs()[0].name: inp})[0].flatten()

# 8. CAMERA CAPTURE
def capture_face():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Camera error")
        return None
    
    print("Smile! Capturing in 3...")
    time.sleep(3)
    
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print("❌ Capture failed")
        return None
    
    save_path = os.path.join(CONTENT_DIR, 'captured.jpg')
    cv2.imwrite(save_path, frame)
    return save_path

# 9. MAIN SYSTEM
print("🎉 Welcome to Face Recognition Attendance!")
print(f"Base directory: {BASE_DIR}\n")

# Verify files and auto-download missing ones
verify_files()

try:
    # Load models
    session = ort.InferenceSession(ARCFACE_PATH)
    face_net = cv2.dnn.readNetFromCaffe(PROTO_PATH, CAFFEMDL_PATH)
    
    # Load known faces
    known = {}
    print("\nLoading known faces...")
    for file in os.listdir(KNOWN_FACES_DIR):
        if file.lower().endswith(('.jpg','.jpeg','.png')):
            emb = get_embedding(os.path.join(KNOWN_FACES_DIR, file))
            if emb is not None:
                known[os.path.splitext(file)[0]] = emb
                print(f"✅ Loaded: {file}")
    
    if not known:
        print("⚠ No faces found in known_faces folder!")
        print(f"Please add student images to: {KNOWN_FACES_DIR}")
    
    # Capture new face
    print("\nStarting face capture...")
    captured = capture_face()
    
    if captured:
        print("Processing image...")
        emb = get_embedding(captured)
        
        if emb is not None:
            # Find match
            for name, known_emb in known.items():
                similarity = cosine_similarity([emb], [known_emb])[0][0]
                print(f"{name}: {similarity*100:.1f}% match")
                
                if similarity >= 0.5:  # Threshold
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    
                    # Save to CSV
                    with open(ATTENDANCE_PATH, 'a', newline='') as f:
                        writer = csv.writer(f)
                        if os.stat(ATTENDANCE_PATH).st_size == 0:
                            writer.writerow(['Name','Time'])
                        writer.writerow([name, timestamp])
                    
                    speak(f"Welcome {name}, attendance marked!")
                    send_email(name, timestamp)
                    break
            else:
                print("⚠ No match found")
                speak("Sorry, not recognized")
        
except Exception as e:
    print(f"⚠ System error: {e}")
finally:
    print("\nSystem shutdown")

🎉 Welcome to Face Recognition Attendance!
Base directory: E:\sameer makhani\attendance

🔍 Verifying Files:
✅ ArcFace Model: E:\sameer makhani\attendance\content\arcface.onnx
✅ Prototxt File: E:\sameer makhani\attendance\deploy.prototxt
✅ Caffe Model: E:\sameer makhani\attendance\res10_300x300_ssd_iter_140000.caffemodel
✅ Known Faces Folder: E:\sameer makhani\attendance\content\known_faces
⚠ System error: [ONNXRuntimeError] : 7 : INVALID_PROTOBUF : Load model from E:\sameer makhani\attendance\content\arcface.onnx failed:Protobuf parsing failed.

System shutdown


In [28]:
import onnx
model = onnx.load("E:/sameer makhani/attendance/content/arcface.onnx")
onnx.checker.check_model(model)



ModuleNotFoundError: No module named 'onnx'

In [29]:
pip install onnx


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
    --------------------------------------- 0.3/15.9 MB ? eta -:--:--
    --------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
import onnx
model = onnx.load("E:/sameer makhani/attendance/content/arcface.onnx")
onnx.checker.check_model(model)


DecodeError: Error parsing message with type 'onnx.ModelProto'

In [31]:
import onnx
model = onnx.load("E:/sameer makhani/attendance/content/arcface.onnx")
onnx.checker.check_model(model)
print("✅ Model is valid.")


✅ Model is valid.


In [36]:
# attendancecode.ipynb - Face Recognition Attendance System
# FULLY UPDATED FOR YOUR E:\sameer makhani\attendance STRUCTURE

# 1. INSTALL DEPENDENCIES (Run this first)
# !pip install opencv-python numpy onnxruntime gTTS scikit-learn Pillow python-dotenv

# 2. IMPORTS
import os
import cv2
import numpy as np
import csv
import smtplib
import ssl
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import onnxruntime as ort
from gtts import gTTS
from email.message import EmailMessage
import time
import platform
import urllib.request

# 3. PATH CONFIGURATION (UPDATED FOR YOUR STRUCTURE)
BASE_DIR = r"E:\sameer makhani\attendance"  # RAW STRING FOR WINDOWS
CONTENT_DIR = os.path.join(BASE_DIR, 'content')
KNOWN_FACES_DIR = os.path.join(CONTENT_DIR, 'known_faces')
ATTENDANCE_PATH = os.path.join(CONTENT_DIR, 'attendance.csv')
ARCFACE_PATH = os.path.join(CONTENT_DIR, 'arcface.onnx')
PROTO_PATH = os.path.join(BASE_DIR, 'deploy.prototxt')
CAFFEMDL_PATH = os.path.join(BASE_DIR, 'res10_300x300_ssd_iter_140000.caffemodel')

# 4. VERIFY FILES EXIST
def verify_files():
    required_files = {
        "ArcFace Model": ARCFACE_PATH,
        "Prototxt File": PROTO_PATH,
        "Caffe Model": CAFFEMDL_PATH,
        "Known Faces Folder": KNOWN_FACES_DIR
    }
    
    print("🔍 Verifying Files:")
    all_exists = True
    for name, path in required_files.items():
        exists = os.path.exists(path)
        print(f"{'✅' if exists else '❌'} {name}: {path}")
        if not exists: all_exists = False
    
    if not all_exists:
        print("\n⚠ Missing files detected. Attempting to download models...")
        download_models()

def download_models():
    # Download ArcFace model if missing
    if not os.path.exists(ARCFACE_PATH):
        print("Downloading ArcFace model...")
        try:
            urllib.request.urlretrieve(
                "https://storage.googleapis.com/ailia-models/arcface/arcfaceresnet100-8.onnx",
                ARCFACE_PATH
            )
            print(f"✅ ArcFace model saved to {ARCFACE_PATH}")
        except Exception as e:
            print(f"❌ Download failed: {e}")

    # Download Caffe files if missing
    if not os.path.exists(PROTO_PATH):
        print("Downloading deploy.prototxt...")
        urllib.request.urlretrieve(
            "https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt",
            PROTO_PATH
        )
    
    if not os.path.exists(CAFFEMDL_PATH):
        print("Downloading Caffe model...")
        urllib.request.urlretrieve(
            "https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel",
            CAFFEMDL_PATH
        )

# 5. AUDIO FEEDBACK
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("voice.mp3")
    
    system = platform.system()
    if system == "Windows":
        os.system("start voice.mp3")
    elif system == "Darwin":  # Mac
        os.system("afplay voice.mp3")
    else:  # Linux
        os.system("mpg123 voice.mp3")
    time.sleep(2)  # Allow audio to play

# 6. EMAIL FUNCTION
def send_email(name, timestamp):
    try:
        msg = EmailMessage()
        msg['Subject'] = f'Attendance Marked: {name}'
        msg['From'] = 'system@institute.com'
        msg['To'] = 'businessoftshirts@gmail.com'
        msg.set_content(f'Student: {name}\nTime: {timestamp}')

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login('sam.makhani33@gmail.com', 'hpfi jhnc eixw udzs')
            server.send_message(msg)
        print("✅ Email sent")
    except Exception as e:
        print(f"⚠ Email failed: {e}")

# 7. FACE RECOGNITION CORE
def get_embedding(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠ Couldn't load image: {img_path}")
        return None
        
    # Face detection
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300,300)), 1.0, (300,300), (104,117,123))
    face_net.setInput(blob)
    detections = face_net.forward()
    
    if detections.shape[2] == 0 or detections[0,0,0,2] < 0.5:
        print(f"⚠ No face found in: {img_path}")
        return None
        
    # Extract face
    h, w = img.shape[:2]
    box = detections[0,0,0,3:7] * np.array([w,h,w,h])
    (x1, y1, x2, y2) = box.astype("int")
    face = img[y1:y2, x1:x2]
    
    # Get embedding
    face = cv2.resize(face, (112,112))
    rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB).astype(np.float32)
    norm = (rgb/255.0 - 0.5)/0.5
    inp = np.transpose(norm, (2,0,1))[np.newaxis,:]
    return session.run(None, {session.get_inputs()[0].name: inp})[0].flatten()

# 8. CAMERA CAPTURE
def capture_face():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Camera error")
        return None
    
    print("Smile! Capturing in 3...")
    time.sleep(3)
    
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print("❌ Capture failed")
        return None
    
    save_path = os.path.join(CONTENT_DIR, 'captured.jpg')
    cv2.imwrite(save_path, frame)
    return save_path

# 9. MAIN SYSTEM
print("🎉 Welcome to Face Recognition Attendance!")
print(f"Base directory: {BASE_DIR}\n")

# Verify files and auto-download missing ones
verify_files()

try:
    # Load models
    session = ort.InferenceSession(ARCFACE_PATH)
    face_net = cv2.dnn.readNetFromCaffe(PROTO_PATH, CAFFEMDL_PATH)
    
    # # Load known faces
    # Load known faces from cache or compute once
    known_faces_file = os.path.join(CONTENT_DIR, "known_faces.npz")
    known = {}

    if os.path.exists(known_faces_file):
        print("✅ Loading cached face embeddings...")
        data = np.load(known_faces_file, allow_pickle=True)
        names = data['names']
        embeddings = data['embeddings']
        known = {name: emb for name, emb in zip(names, embeddings)}
    else:
        print("\n⚙ Creating face embeddings (only once)...")
        names = []
        embeddings = []
    for file in os.listdir(KNOWN_FACES_DIR):
        if file.lower().endswith(('.jpg','.jpeg','.png')):
            img_path = os.path.join(KNOWN_FACES_DIR, file)
            emb = get_embedding(img_path)
            if emb is not None:
                name = os.path.splitext(file)[0]
                known[name] = emb
                names.append(name)
                embeddings.append(emb)
                print(f"✅ Loaded: {file}")

    if known:
        np.savez(known_faces_file, names=names, embeddings=embeddings)
        print(f"✅ Embeddings saved to {known_faces_file}")
    else:
        print("⚠ No valid faces found to cache.")

    # known = {}
    # print("\nLoading known faces...")
    # for file in os.listdir(KNOWN_FACES_DIR):
    #     if file.lower().endswith(('.jpg','.jpeg','.png')):
    #         emb = get_embedding(os.path.join(KNOWN_FACES_DIR, file))
    #         if emb is not None:
    #             known[os.path.splitext(file)[0]] = emb
    #             print(f"✅ Loaded: {file}")
    
    # if not known:
    #     print("⚠ No faces found in known_faces folder!")
    #     print(f"Please add student images to: {KNOWN_FACES_DIR}")
    
    # # Capture new face
    # print("\nStarting face capture...")
    # captured = capture_face()
    
    if captured:
        print("Processing image...")
        emb = get_embedding(captured)
        
        if emb is not None:
            # Find match
            for name, known_emb in known.items():
                similarity = cosine_similarity([emb], [known_emb])[0][0]
                print(f"{name}: {similarity*100:.1f}% match")
                
                if similarity >= 0.5:  # Threshold
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    
                    # Save to CSV
                    with open(ATTENDANCE_PATH, 'a', newline='') as f:
                        writer = csv.writer(f)
                        if os.stat(ATTENDANCE_PATH).st_size == 0:
                            writer.writerow(['Name','Time'])
                        writer.writerow([name, timestamp])
                    
                    speak(f"Welcome {name}, attendance marked!")
                    send_email(name, timestamp)
                    break
            else:
                print("⚠ No match found")
                speak("Sorry, not recognized")
        
except Exception as e:
    print(f"⚠ System error: {e}")
finally:
    print("\nSystem shutdown")

🎉 Welcome to Face Recognition Attendance!
Base directory: E:\sameer makhani\attendance

🔍 Verifying Files:
✅ ArcFace Model: E:\sameer makhani\attendance\content\arcface.onnx
✅ Prototxt File: E:\sameer makhani\attendance\deploy.prototxt
✅ Caffe Model: E:\sameer makhani\attendance\res10_300x300_ssd_iter_140000.caffemodel
✅ Known Faces Folder: E:\sameer makhani\attendance\content\known_faces

⚙ Creating face embeddings (only once)...
✅ Loaded: fizza.jpg
✅ Loaded: razib.jpg
✅ Loaded: saad.jpg
✅ Loaded: sameer.png
✅ Loaded: sufyan.jpg
✅ Embeddings saved to E:\sameer makhani\attendance\content\known_faces.npz
Processing image...
fizza: -0.6% match
razib: 4.9% match
saad: 1.9% match
sameer: 41.6% match
sufyan: 4.3% match
⚠ No match found

System shutdown


In [37]:
pip install ipywidgets


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.2 MB 280.0 kB/s eta 0:00:06
   --------- ------------------------------ 0.5/2.2 MB 280.0 kB/s eta 0:00:06
   --------- ------------------------------ 0.5/2.2 MB 280.0 kB/s eta 0:00:0


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# attendancecode.ipynb - Face Recognition Attendance System
# FULLY UPDATED FOR YOUR E:\sameer makhani\attendance STRUCTURE

# 1. INSTALL DEPENDENCIES (Run this first)
# !pip install opencv-python numpy onnxruntime gTTS scikit-learn Pillow python-dotenv

# 2. IMPORTS
import os
import cv2
import numpy as np
import csv
import smtplib
import ssl
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import onnxruntime as ort
from gtts import gTTS
from email.message import EmailMessage
import time
import platform
from ipywidgets import Button, VBox, Output
from IPython.display import display

import urllib.request

# 3. PATH CONFIGURATION (UPDATED FOR YOUR STRUCTURE)
BASE_DIR = r"E:\sameer makhani\attendance"  # RAW STRING FOR WINDOWS
CONTENT_DIR = os.path.join(BASE_DIR, 'content')
KNOWN_FACES_DIR = os.path.join(CONTENT_DIR, 'known_faces')
ATTENDANCE_PATH = os.path.join(CONTENT_DIR, 'attendance.csv')
ARCFACE_PATH = os.path.join(CONTENT_DIR, 'arcface.onnx')
PROTO_PATH = os.path.join(BASE_DIR, 'deploy.prototxt')
CAFFEMDL_PATH = os.path.join(BASE_DIR, 'res10_300x300_ssd_iter_140000.caffemodel')

# 4. VERIFY FILES EXIST
def verify_files():
    required_files = {
        "ArcFace Model": ARCFACE_PATH,
        "Prototxt File": PROTO_PATH,
        "Caffe Model": CAFFEMDL_PATH,
        "Known Faces Folder": KNOWN_FACES_DIR
    }
    
    print("🔍 Verifying Files:")
    all_exists = True
    for name, path in required_files.items():
        exists = os.path.exists(path)
        print(f"{'✅' if exists else '❌'} {name}: {path}")
        if not exists: all_exists = False
    
    if not all_exists:
        print("\n⚠ Missing files detected. Attempting to download models...")
        download_models()

def download_models():
    # Download ArcFace model if missing
    if not os.path.exists(ARCFACE_PATH):
        print("Downloading ArcFace model...")
        try:
            urllib.request.urlretrieve(
                "https://storage.googleapis.com/ailia-models/arcface/arcfaceresnet100-8.onnx",
                ARCFACE_PATH
            )
            print(f"✅ ArcFace model saved to {ARCFACE_PATH}")
        except Exception as e:
            print(f"❌ Download failed: {e}")

    # Download Caffe files if missing
    if not os.path.exists(PROTO_PATH):
        print("Downloading deploy.prototxt...")
        urllib.request.urlretrieve(
            "https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt",
            PROTO_PATH
        )
    
    if not os.path.exists(CAFFEMDL_PATH):
        print("Downloading Caffe model...")
        urllib.request.urlretrieve(
            "https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel",
            CAFFEMDL_PATH
        )

# 5. AUDIO FEEDBACK
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("voice.mp3")
    
    system = platform.system()
    if system == "Windows":
        os.system("start voice.mp3")
    elif system == "Darwin":  # Mac
        os.system("afplay voice.mp3")
    else:  # Linux
        os.system("mpg123 voice.mp3")
    time.sleep(2)  # Allow audio to play

# 6. EMAIL FUNCTION
def send_email(name, timestamp):
    try:
        msg = EmailMessage()
        msg['Subject'] = f'Attendance Marked: {name}'
        msg['From'] = 'system@institute.com'
        msg['To'] = 'businessoftshirts@gmail.com'
        msg.set_content(f'Student: {name}\nTime: {timestamp}')

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login('sam.makhani33@gmail.com', 'hpfi jhnc eixw udzs')
            server.send_message(msg)
        print("✅ Email sent")
    except Exception as e:
        print(f"⚠ Email failed: {e}")

# 7. FACE RECOGNITION CORE
def get_embedding(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠ Couldn't load image: {img_path}")
        return None
        
    # Face detection
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300,300)), 1.0, (300,300), (104,117,123))
    face_net.setInput(blob)
    detections = face_net.forward()
    
    if detections.shape[2] == 0 or detections[0,0,0,2] < 0.5:
        print(f"⚠ No face found in: {img_path}")
        return None
        
    # Extract face
    h, w = img.shape[:2]
    box = detections[0,0,0,3:7] * np.array([w,h,w,h])
    (x1, y1, x2, y2) = box.astype("int")
    face = img[y1:y2, x1:x2]
    
    # Get embedding
    face = cv2.resize(face, (112,112))
    rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB).astype(np.float32)
    norm = (rgb/255.0 - 0.5)/0.5
    inp = np.transpose(norm, (2,0,1))[np.newaxis,:]
    return session.run(None, {session.get_inputs()[0].name: inp})[0].flatten()

# 8. CAMERA CAPTURE
def capture_face():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Camera error")
        return None
    
    print("Smile! Capturing in 3...")
    time.sleep(3)
    
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print("❌ Capture failed")
        return None
    
    save_path = os.path.join(CONTENT_DIR, 'captured.jpg')
    cv2.imwrite(save_path, frame)
    return save_path

# 9. MAIN SYSTEM
print("🎉 Welcome to Face Recognition Attendance!")
print(f"Base directory: {BASE_DIR}\n")

# Verify files and auto-download missing ones
verify_files()

try:
    # Load models
    session = ort.InferenceSession(ARCFACE_PATH)
    face_net = cv2.dnn.readNetFromCaffe(PROTO_PATH, CAFFEMDL_PATH)
    
    # --- Load or Refresh Known Faces ---

    refresh_button = Button(description="🔄 Refresh Known Faces Cache")
    output = Output()

    def load_known_faces(force_refresh=False):
        known = {}
        known_faces_file = os.path.join(CONTENT_DIR, "known_faces.npz")

        if os.path.exists(known_faces_file) and not force_refresh:
            print("✅ Loading cached face embeddings...")
            data = np.load(known_faces_file, allow_pickle=True)
            names = data['names']
            embeddings = data['embeddings']
            known = {name: emb for name, emb in zip(names, embeddings)}
        else:
            print("⚙ Rebuilding face embeddings...")
            names, embeddings = [], []
            for file in os.listdir(KNOWN_FACES_DIR):
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    path = os.path.join(KNOWN_FACES_DIR, file)
                    emb= get_embedding(path)
                    if emb is not None:
                        name = os.path.splitext(file)[0]
                        known[name] = emb
                        names.append(name)
                        embeddings.append(emb)
                        print(f"✅ Processed: {file}")
            if known:
                np.savez(known_faces_file, names=names, embeddings=embeddings)
                print(f"✅ Embeddings saved to {known_faces_file}")
            else:
                print("⚠ No valid faces found.")

        return known

# Button callback
def on_refresh_click(b):
    with output:
        print("🔁 Refreshing known faces cache...")
        global known
        known = load_known_faces(force_refresh=True)

# Attach callback
refresh_button.on_click(on_refresh_click)

# Display the button
display(VBox([refresh_button, output]))

# Initial load (cached if available)
known = load_known_faces()


    # known = {}
    # print("\nLoading known faces...")
    # for file in os.listdir(KNOWN_FACES_DIR):
    #     if file.lower().endswith(('.jpg','.jpeg','.png')):
    #         emb = get_embedding(os.path.join(KNOWN_FACES_DIR, file))
    #         if emb is not None:
    #             known[os.path.splitext(file)[0]] = emb
    #             print(f"✅ Loaded: {file}")
    
    # if not known:
    #     print("⚠ No faces found in known_faces folder!")
    #     print(f"Please add student images to: {KNOWN_FACES_DIR}")
    
    # # Capture new face
    # print("\nStarting face capture...")
    # captured = capture_face()
    
    if captured:
        print("Processing image...")
        emb = get_embedding(captured)
        
        if emb is not None:
            # Find match
            for name, known_emb in known.items():
                similarity = cosine_similarity([emb], [known_emb])[0][0]
                print(f"{name}: {similarity*100:.1f}% match")
                
                if similarity >= 0.5:  # Threshold
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    
                    # Save to CSV
                    with open(ATTENDANCE_PATH, 'a', newline='') as f:
                        writer = csv.writer(f)
                        if os.stat(ATTENDANCE_PATH).st_size == 0:
                            writer.writerow(['Name','Time'])
                        writer.writerow([name, timestamp])
                    
                    speak(f"Welcome {name}, attendance marked!")
                    send_email(name, timestamp)
                    break
            else:
                print("⚠ No match found")a
                speak("Sorry, not recognized")
        
except Exception as e:
    print(f"⚠ System error: {e}")
finally:
    print("\nSystem shutdown")

SyntaxError: expected 'except' or 'finally' block (3128867281.py, line 213)

In [ ]:
# attendancecode.ipynb - Face Recognition Attendance System
# FULLY UPDATED FOR YOUR E:\sameer makhani\attendance STRUCTURE

# 1. INSTALL DEPENDENCIES (Run this first)
# !pip install opencv-python numpy onnxruntime gTTS scikit-learn Pillow python-dotenv ipywidgets

# 2. IMPORTS
import os
import cv2
import numpy as np
import csv
import smtplib
import ssl
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import onnxruntime as ort
from gtts import gTTS
from email.message import EmailMessage
import time
import platform
from ipywidgets import Button, VBox, Output
from IPython.display import display
import urllib.request

# 3. PATH CONFIGURATION
BASE_DIR = r"E:\sameer makhani\attendance"
CONTENT_DIR = os.path.join(BASE_DIR, 'content')
KNOWN_FACES_DIR = os.path.join(CONTENT_DIR, 'known_faces')
ATTENDANCE_PATH = os.path.join(CONTENT_DIR, 'attendance.csv')
ARCFACE_PATH = os.path.join(CONTENT_DIR, 'arcface.onnx')
PROTO_PATH = os.path.join(BASE_DIR, 'deploy.prototxt')
CAFFEMDL_PATH = os.path.join(BASE_DIR, 'res10_300x300_ssd_iter_140000.caffemodel')

# 4. VERIFY FILES EXIST
def verify_files():
    required_files = {
        "ArcFace Model": ARCFACE_PATH,
        "Prototxt File": PROTO_PATH,
        "Caffe Model": CAFFEMDL_PATH,
        "Known Faces Folder": KNOWN_FACES_DIR
    }
    print("\U0001F50D Verifying Files:")
    all_exists = True
    for name, path in required_files.items():
        exists = os.path.exists(path)
        print(f"{'\u2705' if exists else '\u274C'} {name}: {path}")
        if not exists:
            all_exists = False

    if not all_exists:
        print("\n\u26A0 Missing files detected. Attempting to download models...")
        download_models()

def download_models():
    if not os.path.exists(ARCFACE_PATH):
        print("Downloading ArcFace model...")
        try:
            urllib.request.urlretrieve(
                "https://storage.googleapis.com/ailia-models/arcface/arcfaceresnet100-8.onnx",
                ARCFACE_PATH
            )
            print(f"\u2705 ArcFace model saved to {ARCFACE_PATH}")
        except Exception as e:
            print(f"\u274C Download failed: {e}")

    if not os.path.exists(PROTO_PATH):
        print("Downloading deploy.prototxt...")
        urllib.request.urlretrieve(
            "https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt",
            PROTO_PATH
        )

    if not os.path.exists(CAFFEMDL_PATH):
        print("Downloading Caffe model...")
        urllib.request.urlretrieve(
            "https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel",
            CAFFEMDL_PATH
        )

# 5. AUDIO FEEDBACK
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("voice.mp3")
    system = platform.system()
    if system == "Windows":
        os.system("start voice.mp3")
    elif system == "Darwin":
        os.system("afplay voice.mp3")
    else:
        os.system("mpg123 voice.mp3")
    time.sleep(2)

# 6. EMAIL FUNCTION
def send_email(name, timestamp):
    try:
        msg = EmailMessage()
        msg['Subject'] = f'Attendance Marked: {name}'
        msg['From'] = 'system@institute.com'
        msg['To'] = 'businessoftshirts@gmail.com'
        msg.set_content(f'Student: {name}\nTime: {timestamp}')

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login('sam.makhani33@gmail.com', 'hpfi jhnc eixw udzs')
            server.send_message(msg)
        print("\u2705 Email sent")
    except Exception as e:
        print(f"\u26A0 Email failed: {e}")

# 7. FACE RECOGNITION CORE
def get_embedding(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"\u26A0 Couldn't load image: {img_path}")
        return None

    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300,300)), 1.0, (300,300), (104,117,123))
    face_net.setInput(blob)
    detections = face_net.forward()

    if detections.shape[2] == 0 or detections[0,0,0,2] < 0.5:
        print(f"\u26A0 No face found in: {img_path}")
        return None

    h, w = img.shape[:2]
    box = detections[0,0,0,3:7] * np.array([w,h,w,h])
    (x1, y1, x2, y2) = box.astype("int")
    face = img[y1:y2, x1:x2]
    face = cv2.resize(face, (112,112))
    rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB).astype(np.float32)
    norm = (rgb/255.0 - 0.5)/0.5
    inp = np.transpose(norm, (2,0,1))[np.newaxis,:]
    return session.run(None, {session.get_inputs()[0].name: inp})[0].flatten()

# 8. CAMERA CAPTURE
def capture_face():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("\u274C Camera error")
        return None

    print("Smile! Capturing in 3...")
    time.sleep(3)
    ret, frame = cap.read()
    cap.release()

    if not ret:
        print("\u274C Capture failed")
        return None

    save_path = os.path.join(CONTENT_DIR, 'captured.jpg')
    cv2.imwrite(save_path, frame)
    cv2.imshow("Captured Image", frame)
    cv2.waitKey(2000)
    cv2.destroyAllWindows()
    return save_path

# 9. MAIN SYSTEM
print("\U0001F389 Welcome to Face Recognition Attendance!")
print(f"Base directory: {BASE_DIR}\n")
verify_files()

try:
    session = ort.InferenceSession(ARCFACE_PATH)
    face_net = cv2.dnn.readNetFromCaffe(PROTO_PATH, CAFFEMDL_PATH)

    refresh_button = Button(description="\U0001F501 Refresh Known Faces Cache")
    output = Output()

    def load_known_faces(force_refresh=False):
        known = {}
        known_faces_file = os.path.join(CONTENT_DIR, "known_faces.npz")

        if os.path.exists(known_faces_file) and not force_refresh:
            print("\u2705 Loading cached face embeddings...")
            data = np.load(known_faces_file, allow_pickle=True)
            names = data['names']
            embeddings = data['embeddings']
            known = {name: emb for name, emb in zip(names, embeddings)}
        else:
            print("\u2699 Rebuilding face embeddings...")
            names, embeddings = [], []
            for file in os.listdir(KNOWN_FACES_DIR):
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    path = os.path.join(KNOWN_FACES_DIR, file)
                    emb = get_embedding(path)
                    if emb is not None:
                        name = os.path.splitext(file)[0]
                        known[name] = emb
                        names.append(name)
                        embeddings.append(emb)
                        print(f"\u2705 Processed: {file}")
            if known:
                np.savez(known_faces_file, names=names, embeddings=embeddings)
                print(f"\u2705 Embeddings saved to {known_faces_file}")
            else:
                print("\u26A0 No valid faces found.")

        return known

    def on_refresh_click(b):
        with output:
            print("\U0001F501 Refreshing known faces cache...")
            global known
            known = load_known_faces(force_refresh=True)

    refresh_button.on_click(on_refresh_click)
    display(VBox([refresh_button, output]))

    known = load_known_faces()
    captured = capture_face()

    if captured:
        print("Processing image...")
        emb = get_embedding(captured)
        if emb is not None:
            for name, known_emb in known.items():
                similarity = cosine_similarity([emb], [known_emb])[0][0]
                print(f"{name}: {similarity*100:.1f}% match")
                if similarity >= 0.5:
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    with open(ATTENDANCE_PATH, 'a', newline='') as f:
                        writer = csv.writer(f)
                        if os.stat(ATTENDANCE_PATH).st_size == 0:
                            writer.writerow(['Name','Time'])
                        writer.writerow([name, timestamp])
                    speak(f"Welcome {name}, attendance marked!")
                    send_email(name, timestamp)
                    break
            else:
                print("\u26A0 No match found")
                speak("Sorry, not recognized")

except Exception as e:
    print(f"\u26A0 System error: {e}")
finally:
    print("\nSystem shutdown")


🎉 Welcome to Face Recognition Attendance!
Base directory: E:\sameer makhani\attendance

🔍 Verifying Files:
✅ ArcFace Model: E:\sameer makhani\attendance\content\arcface.onnx
✅ Prototxt File: E:\sameer makhani\attendance\deploy.prototxt
✅ Caffe Model: E:\sameer makhani\attendance\res10_300x300_ssd_iter_140000.caffemodel
✅ Known Faces Folder: E:\sameer makhani\attendance\content\known_faces


✅ Loading cached face embeddings...
Smile! Capturing in 3...
Processing image...
fizza: -6.9% match
razib: 14.6% match
saad: 8.1% match
sameer: 48.7% match
sufyan: 1.1% match
⚠ No match found

System shutdown


In [ ]:
# attendancecode.ipynb - Face Recognition Attendance System
# FULLY UPDATED FOR YOUR E:\sameer makhani\attendance STRUCTURE

# 1. INSTALL DEPENDENCIES (Run this first)
# !pip install opencv-python numpy onnxruntime gTTS scikit-learn Pillow python-dotenv ipywidgets

# 2. IMPORTS
import os
import cv2
import numpy as np
import csv
import smtplib
import ssl
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import onnxruntime as ort
from gtts import gTTS
from email.message import EmailMessage
import time
import platform
from ipywidgets import Button, VBox, Output
from IPython.display import display
import urllib.request

# 3. PATH CONFIGURATION
BASE_DIR = r"E:\sameer makhani\attendance"
CONTENT_DIR = os.path.join(BASE_DIR, 'content')
KNOWN_FACES_DIR = os.path.join(CONTENT_DIR, 'known_faces')
ATTENDANCE_PATH = os.path.join(CONTENT_DIR, 'attendance.csv')
ARCFACE_PATH = os.path.join(CONTENT_DIR, 'arcface.onnx')
PROTO_PATH = os.path.join(BASE_DIR, 'deploy.prototxt')
CAFFEMDL_PATH = os.path.join(BASE_DIR, 'res10_300x300_ssd_iter_140000.caffemodel')

# 4. VERIFY FILES EXIST
def verify_files():
    required_files = {
        "ArcFace Model": ARCFACE_PATH,
        "Prototxt File": PROTO_PATH,
        "Caffe Model": CAFFEMDL_PATH,
        "Known Faces Folder": KNOWN_FACES_DIR
    }
    print("\U0001F50D Verifying Files:")
    all_exists = True
    for name, path in required_files.items():
        exists = os.path.exists(path)
        print(f"{'\u2705' if exists else '\u274C'} {name}: {path}")
        if not exists:
            all_exists = False

    if not all_exists:
        print("\n\u26A0 Missing files detected. Attempting to download models...")
        download_models()

def download_models():
    if not os.path.exists(ARCFACE_PATH):
        print("Downloading ArcFace model...")
        try:
            urllib.request.urlretrieve(
                "https://storage.googleapis.com/ailia-models/arcface/arcfaceresnet100-8.onnx",
                ARCFACE_PATH
            )
            print(f"\u2705 ArcFace model saved to {ARCFACE_PATH}")
        except Exception as e:
            print(f"\u274C Download failed: {e}")

    if not os.path.exists(PROTO_PATH):
        print("Downloading deploy.prototxt...")
        urllib.request.urlretrieve(
            "https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt",
            PROTO_PATH
        )

    if not os.path.exists(CAFFEMDL_PATH):
        print("Downloading Caffe model...")
        urllib.request.urlretrieve(
            "https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel",
            CAFFEMDL_PATH
        )

# 5. AUDIO FEEDBACK
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("voice.mp3")
    system = platform.system()
    if system == "Windows":
        os.system("start voice.mp3")
    elif system == "Darwin":
        os.system("afplay voice.mp3")
    else:
        os.system("mpg123 voice.mp3")
    time.sleep(2)

# 6. EMAIL FUNCTION
def send_email(name, timestamp):
    try:
        msg = EmailMessage()
        msg['Subject'] = f'Attendance Marked: {name}'
        msg['From'] = 'system@institute.com'
        msg['To'] = 'businessoftshirts@gmail.com'
        msg.set_content(f'Student: {name}\nTime: {timestamp}')

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login('sam.makhani33@gmail.com', 'hpfi jhnc eixw udzs')
            server.send_message(msg)
        print("\u2705 Email sent")
    except Exception as e:
        print(f"\u26A0 Email failed: {e}")

# 7. FACE RECOGNITION CORE
def get_embedding(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"\u26A0 Couldn't load image: {img_path}")
        return None

    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300,300)), 1.0, (300,300), (104,117,123))
    face_net.setInput(blob)
    detections = face_net.forward()

    if detections.shape[2] == 0 or detections[0,0,0,2] < 0.5:
        print(f"\u26A0 No face found in: {img_path}")
        return None

    h, w = img.shape[:2]
    box = detections[0,0,0,3:7] * np.array([w,h,w,h])
    (x1, y1, x2, y2) = box.astype("int")
    face = img[y1:y2, x1:x2]
    face = cv2.resize(face, (112,112))
    rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB).astype(np.float32)
    norm = (rgb/255.0 - 0.5)/0.5
    inp = np.transpose(norm, (2,0,1))[np.newaxis,:]
    return session.run(None, {session.get_inputs()[0].name: inp})[0].flatten()

# 8. CAMERA CAPTURE
def capture_face():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("\u274C Camera error")
        return None

    print("Smile! Capturing in 3...")
    time.sleep(3)
    ret, frame = cap.read()
    cap.release()

    if not ret:
        print("\u274C Capture failed")
        return None

    save_path = os.path.join(CONTENT_DIR, 'captured.jpg')
    cv2.imwrite(save_path, frame)
    cv2.imshow("Captured Image", frame)
    cv2.waitKey(2000)
    cv2.destroyAllWindows()
    return save_path

# 9. MAIN SYSTEM
print("\U0001F389 Welcome to Face Recognition Attendance!")
print(f"Base directory: {BASE_DIR}\n")
verify_files()

try:
    session = ort.InferenceSession(ARCFACE_PATH)
    face_net = cv2.dnn.readNetFromCaffe(PROTO_PATH, CAFFEMDL_PATH)

    refresh_button = Button(description="\U0001F501 Refresh Known Faces Cache")
    output = Output()

    def load_known_faces(force_refresh=False):
        known = {}
        known_faces_file = os.path.join(CONTENT_DIR, "known_faces.npz")

        if os.path.exists(known_faces_file) and not force_refresh:
            print("\u2705 Loading cached face embeddings...")
            data = np.load(known_faces_file, allow_pickle=True)
            names = data['names']
            embeddings = data['embeddings']
            known = {name: emb for name, emb in zip(names, embeddings)}
        else:
            print("\u2699 Rebuilding face embeddings...")
            names, embeddings = [], []
            for file in os.listdir(KNOWN_FACES_DIR):
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    path = os.path.join(KNOWN_FACES_DIR, file)
                    emb = get_embedding(path)
                    if emb is not None:
                        name = os.path.splitext(file)[0]
                        known[name] = emb
                        names.append(name)
                        embeddings.append(emb)
                        print(f"\u2705 Processed: {file}")
            if known:
                np.savez(known_faces_file, names=names, embeddings=embeddings)
                print(f"\u2705 Embeddings saved to {known_faces_file}")
            else:
                print("\u26A0 No valid faces found.")

        return known

    def on_refresh_click(b):
        with output:
            print("\U0001F501 Refreshing known faces cache...")
            global known
            known = load_known_faces(force_refresh=True)

    refresh_button.on_click(on_refresh_click)
    display(VBox([refresh_button, output]))

    known = load_known_faces()
    captured = capture_face()

    if captured:
        print("Processing image...")
        emb = get_embedding(captured)
        if emb is not None:
            for name, known_emb in known.items():
                similarity = cosine_similarity([emb], [known_emb])[0][0]
                print(f"{name}: {similarity*100:.1f}% match")
                if similarity >= 0.5:
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    with open(ATTENDANCE_PATH, 'a', newline='') as f:
                        writer = csv.writer(f)
                        if os.stat(ATTENDANCE_PATH).st_size == 0:
                            writer.writerow(['Name','Time'])
                        writer.writerow([name, timestamp])
                    speak(f"Welcome {name}, attendance marked!")
                    send_email(name, timestamp)
                    break
            else:
                print("\u26A0 No match found")
                speak("Sorry, not recognized")

except Exception as e:
    print(f"\u26A0 System error: {e}")
finally:
    print("\nSystem shutdown")


In [ ]:
!pip install opencv-python numpy onnxruntime gTTS scikit-learn Pillow ipywidgets
